In [ ]:
import zmq
import time
import traceback
import numpy as np
import struct

In [ ]:
class BoxPublisher:
    def __init__(self):
        context = zmq.Context()
        self.pub = context.socket(zmq.PUB)
        self.pub.setsockopt(zmq.LINGER, 0)
        self.pub.bind("tcp://*:5555")
        self.sub = context.socket(zmq.SUB)
        self.sub.setsockopt(zmq.SUBSCRIBE, b"")
        self.sub.setsockopt(zmq.LINGER, 0)
        self.sub.bind("tcp://*:5556")
        self.active = True

    def close(self):
        self.sub.close()
        self.pub.close()
    
    def publish(self, messages, debug=False):
        if self.active:
            if debug:
                print("Sending messages " + str(messages))
            self.pub.send_multipart(messages)

    def terminated(self, debug=False):
        if self.active:
            try:
                signal = self.sub.recv(flags=zmq.NOBLOCK)
                if debug:
                    print("recvd TERMINATE signal")
                self.active = False
            except zmq.error.Again:
                # signal not sent yet
                pass
            except:
                traceback.print_exc()
        return not self.active

In [ ]:
if __name__ == "__main__":
    pub = BoxPublisher()
    while not pub.terminated():
        time.sleep(1)
        float_arr = np.array([0.1, 0.2, 0.3, 0.4])
        float_list = [float(f) for f in float_arr]
        messages = [struct.pack('d', f) for f in float_list]
        print(float_arr)
        print(float_list)
        print(messages)
        pub.publish(messages, True)
    print("Shutting down")
    pub.close()